In [2]:
from GeometricObjects import *
from dataclasses import dataclass

In [3]:
T = Triangle([Vector(0, 0, 0), Vector(0, 1, 0), Vector(1, 1, 1)])
RESOLUTION = 1


In [4]:
@dataclass
class Hit:
    tri: Triangle
    obj: TriObject
    u: float
    v: float
    t: float
    origin: Vector
    dir: Vector


class Pixel:
    def __init__(self, center: Vector) -> None:
        self.status = 0
        self.center = center


class Triangle:
    def __init__(self, coords: list[Vector]):
        self.a: Vector = coords[0]
        self.b: Vector = coords[1]
        self.c: Vector = coords[2]
        self.at = None
        self.bt = None
        self.ct = None

    def getArea(self) -> float:
        return 1 / 2 * abs(((self.a - self.b).cross(self.a - self.c)))

    def intersect(self, ray_start, ray_vec) -> tuple[bool, Vector]:
        """
        Detect intersection between this triangle and ray_vec originating from ray_start

        :param ray_start: Vector indicating the origin of the ray
        :param ray_vec: Vector indicating direction of the ray
        :return: bool indicating if it was a hit, and vector indicating parameterized t, and local coordinates u, v
        """
        # define a null intersection
        # null_inter = [None, None, None]  # np.array([np.nan, np.nan, np.nan])
        null_inter = Vector(None, None, None)
        # ray_start = np.asarray(ray_start)
        # ray_vec = np.asarray(ray_vec)

        # break down triangle into the individual points
        v1, v2, v3 = self.a, self.b, self.c
        eps = 0.000001

        # compute edges
        edge1 = v2 - v1
        edge2 = v3 - v1
        # pvec = np.cross(ray_vec, edge2)
        pvec = ray_vec.cross(edge2)
        det = edge1.dot(pvec)

        if abs(det) < eps:  # no intersection
            # print('fail1')
            return False, null_inter
        inv_det = 1.0 / det
        tvec = ray_start - v1
        u = tvec.dot(pvec) * inv_det
        # print(u)
        if u < 0.0 or u > 1.0:  # if not intersection
            # print('fail2')
            return False, null_inter

        qvec = tvec.cross(edge1)
        v = ray_vec.dot(qvec) * inv_det
        if v < 0.0 or u + v > 1.0:  # if not intersection
            #  print('fail3')
            return False, null_inter

        t = edge2.dot(qvec) * inv_det
        if t < eps:
            #   print('fail4')
            return False, null_inter

        return True, Vector(t, u, v)


Tri[< 0.0,  0.0,  0.0>, < 0.0,  1.0,  0.0>, < 1.0,  1.0,  1.0>]


In [ ]:
class TriObject:
    bounding_box: list[Vector]

    def __init__(
        self, name: str, triangles: list[Triangle], points: list[Vector]
    ) -> None:
        self.name = name
        self.triangles = triangles
        self.points = points
        self.texture: list[Pixel] = []
        self.boundingBox: list[Vector] = []
        self.calcBoundingBox()

    def surfaceArea(self) -> float:
        SA = 0
        for triangle in self.triangles:
            SA = +triangle.getArea()
        return SA

    def calcBoundingBox(self):
        minX = min([vec.x for vec in self.points])
        minY = min([vec.y for vec in self.points])
        minZ = min([vec.z for vec in self.points])
        maxX = max([vec.x for vec in self.points])
        maxY = max([vec.y for vec in self.points])
        maxZ = max([vec.z for vec in self.points])
        minPt = Vector(minX, minY, minZ)
        maxPt = Vector(maxX, maxY, maxZ)
        self.boundingBox = [minPt, maxPt]
